<a href="https://colab.research.google.com/github/Alexher90/An-lisis-de-Datos-2/blob/master/Solucion_Semana11_SistemasRecomendacionPorContenido_Actividad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MIIA-4203 MODELOS AVANZADOS PARA ANÁLISIS DE DATOS II


# Sistemas de Recomendación basados en contenido

## Actividad 11


### Profesor: Camilo Franco (c.franco31@uniandes.edu.co)

En este cuadernos estudiaremos los sistemas de recomendacion basados en contenido. Seguiremos trabajando con  la base de datos de películas de IMDB (https://www.imdb.com/) 



**Presentado por:**

**Alexander Hernández Páez: 200920588**

**Juan David Cortés: 201728568**

**Wilson Felipe González: 200924943**

**Alexander Camargo: 200911325**

## Introducción

Los recomendadores basados en contenido se construyen a partir de la identificación de ítems que el usuario prefiere, y la búsqueda de ítems similares en función de determinados atributos, como por ejemplo el género, la sinopsis o el reparto (actores, etc). De esta manera, si el usuario tiene unas preferencias específicas sobre un ítem específico, también podría tener preferencia por un ítem *similar*.

Primero carguemos los datos con los que vamos a trabajar:


In [ ]:
# Importamos la biblioteca Pandas
import pandas as pd

# Cargamos los datos de peliculas de la base de datos IMDB
metadata = pd.read_csv('movies_metadata.csv', low_memory=False)

print(metadata.shape)
      
list(metadata)
#metadata.shape

(45466, 24)


['adult',
 'belongs_to_collection',
 'budget',
 'genres',
 'homepage',
 'id',
 'imdb_id',
 'original_language',
 'original_title',
 'overview',
 'popularity',
 'poster_path',
 'production_companies',
 'production_countries',
 'release_date',
 'revenue',
 'runtime',
 'spoken_languages',
 'status',
 'tagline',
 'title',
 'video',
 'vote_average',
 'vote_count']

In [ ]:
# Así se ven los datos
metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


## 2 .Recomendación de peliculas mas populares por genero

Ahora recordemos la recomendación de películas por género de acuerdo con su popularidad, donde calculamos el voto promedio ponderado $\mu_i$, de la $i$-ésima película como:

$$
\mu_i  = \left( \frac{v_i}{v_{max}} \right) R_i 
$$

donde $v_i$ es el número de votos para la $i$-ésima película, $v_{max}$ es el máximo número de votos que recibe la película más popular, y $R$ es el rating promedio de la pelicula.


In [ ]:
import numpy as np
from ast import literal_eval

# trabajamos la informacion por generos
metadata['genres'] = metadata['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

# añadimos la variable del año
metadata['year'] = pd.to_datetime(metadata['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

metadata.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,1995
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,1995


Primero nos quedamos con todos los generos:

In [ ]:
generos = metadata.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
generos.name = 'genre'
gen_md = metadata.drop('genres', axis=1).join(generos)

gen_md.head(3)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


,adult,belongs_to_collection,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,year,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Animation
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Comedy
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1995,Family


Construimos una funcion para un género particular y que tome en cuenta peliculas con un número vmin de votos:

In [ ]:
def rec_gen(genero, vmin):
    df = gen_md[gen_md['genre'] == genero]
    v = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    R = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    m = df['vote_average'].max()
    
    pelisG = df[(df['vote_count'] >= vmin) & (df['vote_count'].notnull()) & (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity', 'overview', 'homepage']]
    pelisG['vote_count'] = pelisG['vote_count'].astype('int')
    pelisG['vote_average'] = pelisG['vote_average'].astype('int')
    
    pelisG['wr'] = v/m * R
    pelisG = pelisG.sort_values('wr', ascending=False).head(250)
    
    return pelisG



Veamos el Top-15 de recomendaciones en Ciencia Ficción:

In [ ]:
k = 15
scifi = rec_gen('Science Fiction', 1000)
scifi.head(15)

,title,year,vote_count,vote_average,popularity,overview,homepage,wr
15480,Inception,2010,14075,8,29.108149,"Cobb, a skilled thief who commits corporate es...",http://inceptionmovie.warnerbros.com/,11260.0
22879,Interstellar,2014,11187,8,32.213481,Interstellar chronicles the adventures of a gr...,http://www.interstellarmovie.net/,8949.6
14551,Avatar,2009,12114,7,185.070892,"In the 22nd century, a paraplegic Marine is di...",http://www.avatarmovie.com/,8479.8
17818,The Avengers,2012,12000,7,89.887648,When an unexpected enemy emerges and threatens...,http://marvel.com/avengers_movie/,8400.0
23753,Guardians of the Galaxy,2014,10014,7,53.291601,"Light years from Earth, 26 years after being a...",http://marvel.com/guardians,7009.8
26553,Mad Max: Fury Road,2015,9629,7,29.36178,An apocalyptic story set in the furthest reach...,http://www.madmaxmovie.com/,6740.3
2458,The Matrix,1999,9079,7,33.366332,"Set in the 22nd century, The Matrix tells the ...",http://www.warnerbros.com/matrix,6355.3
12588,Iron Man,2008,8951,7,22.073099,"After being held captive in an Afghan cave, bi...",http://www.ironmanmovie.com/,6265.7
18244,The Hunger Games,2012,9634,6,20.031667,Every year in the ruins of what was once North...,http://www.thehungergames.movie/,5780.4
26555,Star Wars: The Force Awakens,2015,7993,7,31.626013,Thirty years after defeating the Galactic Empi...,http://www.starwars.com/films/star-wars-episod...,5595.1


Inception e Interstellar aparecen en las dos primeras posiciones. Personalmente me gusta más Interstellar (si quieres ver un agujero negro, esta película es lo mejor que podrás conseguir), pero reconozco que Inception tiene mucho nivel. Podemos seguir refinando este tipo de recomendaciones prestando atención a los distintos atributos que tenemos disponibles sobre las películas. 


## 3. Sistemas de recomendación basados en contenido

Este tipo de sistemas basados en contenido utiliza información específica sobre el ítem o producto de recomendación. Por ejemplo, si no contamos con información del rating de las peliculas pero sabemos que un usuario vió o que le gustó cierta película, podríamos utilizar la descripción o resumen de la película para construir nuevas recomendaciones a partir de peliculas con contenidos *similares*.

A continuación vamos a construir un sistema que recomiende películas en función de sus descripciones o "su trama". Entonces necesitamos calcular funciones de similitud de acuerdo con la descripción linguistica de cada película.

En nuestros datos, la descripción de cada película la encontramos bajo el atributo "overview". Veamos a continuación las tramas de las primeras 5 peliculas recomendadas de Ciencia Ficcion:

In [ ]:
pd.set_option('display.max_colwidth', -1)
scifi[['title', 'overview']].head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,title,overview
15480,Inception,"Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: ""inception"", the implantation of another person's idea into a target's subconscious."
22879,Interstellar,Interstellar chronicles the adventures of a group of explorers who make use of a newly discovered wormhole to surpass the limitations on human space travel and conquer the vast distances involved in an interstellar voyage.
14551,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization."
17818,The Avengers,"When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!"
23753,Guardians of the Galaxy,"Light years from Earth, 26 years after being abducted, Peter Quill finds himself the prime target of a manhunt after discovering an orb wanted by Ronan the Accuser."


### 3.1 Estimación de similitudes y procesamiento de lenguage natural

En primera instancia podemos evaluar las similitudes entre las películas a partir de la descripción linguística de su contenido. Pero, **¿cómo calculamos estas similitudes, o más aun, cómo procesamos los caracteres linguisticos, las palabras y las frases para calcular dichas similitudes?**

A continuación vamos a ver una primera aproximación al análisis de texto a nivel de *términos* o *palabras*. Para ello, vamos a computar el ínidice TF-IDF (del inglés "Term Frequency-Inverse Document Frequency"), el cual se puede entender como una ponderación de la relevancia de los términos encontrados en cada resumen.  

### 3.1.1 Indice TF-IDF
El índice TF-IDF mide la relevancia de un término linguístico ($t$) por cada resumen o documento que estemos analizando ($d$), tomando la frecuencia del término en cada resumen $tf(t,d)$, y multiplicandola por la frecuencia inversa de la ocurrencia del término en la muestra de resumenes $idf(t)$. De esta manera se extrae la importancia/significancia de los términos/palabras como información numérica para la estimación de la similitud entre películas.

Consideremos un conjunto de resumenes ($D$). En este conjunto es de esperar que los artículos linguísticos sean muy comunes (en ingles "a", "the",...), los cuales no ofrecen en verdad información relevante acerca del contenido de una película. Entonces, si fueramos a introducir el conteo de las palabras directamente a nuestro cálculo de las similitudes (o a un clasificador), esos términos más frecuentes añadirían ruido sobre otros términos menos frecuentes pero posiblemente más interesantes (en verdad relevantes para entender el contenido de las películas). 

De esta manera, la frecuencia de un término $t$ en un resumen $d$ está dada por $tf(t,d)$, y el índice $tf-idf(t,d)$ está dado por 

$$tf-idf(t,d)=tf(t,d)\times idf(t)  $$

donde $$ idf(t)=\log \frac{1+n}{1+df(t)}+1 $$

siendo $n$ el número total de resumenes en $D$ y $df(t)$ es el número de resumenes en $D$ que contienen el término $t$.
 
El resultado de los vectores $tf-idf(d)$, de todos los términos en cada documento, son normalizados por la norma Euclideana $L2$, tal que 

$$ tf-idf(t,d)_{norm} = \frac{tf-idf(t,d)}{\sqrt{tf-idf(t_1,d)+...+tf-idf(t_T,d)}}$$

donde $T$ es el número total de términos.

**Por ejemplo**, *si tenemos 3 términos en 3 resumenes, el primer término $t_1$ aparece 3 veces en el primer resumen $d_1$, 2 veces en el segundo resumen $d_2$ y 3 veces en el tercer resumen $d_3$. El segundo término $t_2$ aparece dos vez en el primer resumen $d_1$, y el tercer término $t_3$ solo aparece una vez en el tercer resumen $d_3$. *

*Entonces $df(t_1)=3$, $df(t_2)=1$ y $df(t_3)=1$.*

*Luego, $idf(t_1)=log(4/4)+1=1$, $idf(t_2)=idf(t_3)=1.69$.*

*Por lo tanto, antes de normalizar, $tf-idf(t_1,d_1)=3\times 1=3$, $tf-idf(t_2,d_1)=2\times 1.69=3.38$ y $tf-idf(t_3,d_1)=0\times 1.69=0$.* 

*Tras la normalización, tendriamos que  $tf-idf(d_1)=\frac{(3,3.38,0)}{\sqrt{9+11.42+0}}=(0.66,0.74,0)$*

La biblioteca **scikit-learn** ofrece la clase *TfIdfVectorizer*, la cual produce una matriz TF-IDF de manera sencilla. Entonces, este índice lo calculamos utilizando los parámetros por defecto del transformador `TfidfTransformer`: `TfidfTransform(norm='l2', use_idf=True, smooth_idf=True, sublinear_tf=False)`.


Como resultado, vamos a obtener una matriz cuyas columnas representan la relevancia (TF-IDF) de los términos presentes en los resumenes de cada película. 


In [ ]:
# Utilizamos el TfIdfVectorizer de la biblioteca scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Definimos el objeto TF-IDF. 
# También se podrían remover articulos (comunes) como 'the', 'a' con (stop_words='english')
tfidf = TfidfVectorizer()  

# Reemplazamos valores NaN con espacio vacío
metadata['overview'] = metadata['overview'].fillna('')

# Construimos la matriz TF-IDF ajustando y transfromando los datos
tfidf_mat = tfidf.fit_transform(metadata['overview'])
pd.DataFrame(tfidf_mat).head()
# La salida con las dimensiones de tfidf_matrix
tfidf_mat.shape


(45466, 76132)

### Pregunta 3.1

- Cuántos términos fueron necesarios para describir las peliculas de nuestra base de datos?

- Qué tipo de matriz es `tfidf_mat`?

In [ ]:
# Definimos el objeto TF-IDF. 
# También se podrían remover articulos (comunes) como 'the', 'a' con (stop_words='english')
tfidf = TfidfVectorizer(stop_words='english')  

# Reemplazamos valores NaN con espacio vacío
metadata['overview'] = metadata['overview'].fillna('')

# Construimos la matriz TF-IDF ajustando y transfromando los datos
tfidf_mat = tfidf.fit_transform(metadata['overview'])
pd.DataFrame(tfidf_mat).head()
# La salida con las dimensiones de tfidf_matrix
tfidf_mat.shape

(45466, 75827)

**- Sin excluir artículos comúnes que no proporcionan información (stopwords), se requirieron 76132 términos para describir las sinopsis de las películas. Utilizando el argumento stopwords = True, se requirieron 75827 términos.**

**- tfidf_mat es una matriz comprimida que contiene la codificación de cada término por cada película con su respectivo indicador de relevancia.**

In [ ]:
print(pd.DataFrame(tfidf_mat).head())
print(type(tfidf_mat))

In [ ]:
# veamos la primera pelicula y la representacion tfidf de su sinopsis
print(metadata['overview'][0])
print(tfidf_mat[0,:])

Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.
  (0, 17764)	0.13483149538639247
  (0, 4388)	0.1474882034218405
  (0, 38030)	0.10142919482788751
  (0, 21887)	0.10438761058719498
  (0, 19641)	0.13281884272823927
  (0, 48558)	0.10339358185033234
  (0, 59519)	0.13008016104455086
  (0, 12490)	0.12544427954397822
  (0, 51108)	0.13434817283119177
  (0, 29238)	0.10093917370354445
  (0, 50914)	0.09190797940163035
  (0, 39423)	0.11907123344715953
  (0, 1847)	0.140911774178889
  (0, 58571)	0.1135591886873686
  (0, 38693)	0.20627924682810617
  (0, 9874)	0.5028038686135609
  (0, 9087)	0.10635375129287977
  (0, 7491)	0.12380553184830104
  (0, 56872)	0.111248510865236
  (0, 28729)	0.13311522181618415
  (0, 39012)	0.08718689178959059
  (0, 67874)	0.

### 3.1.2 Cálculo de similitudes

Ahora ya podemos calcular las similitudes entre las peliculas basados en sus resumenes. Podríamos utilizar distintas métricas, como la Euclideana, la correlación de Pearson, o la similitud del coseno. 

Por ejemplo, veamos qué ocurre con la similitud del coseno calculada para todo par de películas. Lo bueno de esta métrica del coseno es que es independiente de la magnitud y mide la dirección de los vectores. De este modo, dos vectores paralelos (con angulo relativo de 0°) tienen una similitud de 1, y dos vectores ortogonales, con un angulo de 90° entre ellos obtienen una similitud de 0.  

La *similitud del coseno* se define para todo $x,y \in [0,1]$ tal que

$$
sim_{cos}(x,y)=\frac{\sum_{i=1}^{n}x_i y_i}{\sqrt{\sum_{i=1}^{n}x_i^2} \sqrt{\sum_{i=1}^{n}y_i^2}}
$$

Como tenemos la matriz de representación vectorizada de las palabras para cada pelicula, el cómputo del producto interno obtiene de manera directa el valor de similitud por coseno. De esta manera, utilizamos el `linear_kernel()` de sklearn en lugar de `cosine_similarities()`.

In [ ]:
# Utilizamos el linear_kernel
from sklearn.metrics.pairwise import linear_kernel

El cálculo de las similitudes para cada par de entre todas las 45466 peliculas y sus 76132 entradas es bastante pesado. Si ejecutamos el código sobre toda la matriz `tfidf_mat` en GoogleColab, debemos utilizar los recursos de RAM en la máquina remota. En nuestra máquina local tomemos un conjunto de peliculas más pequeño.

Por ejemplo, tomemos solamente las peliculas más populares:

In [ ]:
m = metadata['vote_count'].quantile(0.90)
pelis_P = metadata.copy().loc[metadata['vote_count'] >= m]
pelis_P.shape

(4555, 25)

In [ ]:
tfidf_P = tfidf_mat[ pelis_P.index, :]
tfidf_P.shape

(4555, 75827)

In [ ]:
# Calculamos la matriz de similitudes por coseno (para un numero reducido de observaciones)
sim_cos = linear_kernel(tfidf_P, tfidf_P)#], dense_output=False)

In [ ]:
sim_cos

array([[1.        , 0.01504121, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01504121, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01879787, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.01879787, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.02167055],
       [0.        , 0.        , 0.        , ..., 0.        , 0.02167055,
        1.        ]])

### Ejercicio 3.2

Calcule las similitudes entre peliculas utilizando una funcion de similitud distinta.

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances
Euc = euclidean_distances(tfidf_P, tfidf_P)
Euc

array([[0.        , 1.40353752, 1.41421356, ..., 1.41421356, 1.41421356,
        1.41421356],
       [1.40353752, 0.        , 1.41421356, ..., 1.41421356, 1.41421356,
        1.41421356],
       [1.41421356, 1.41421356, 0.        , ..., 1.4008584 , 1.41421356,
        1.41421356],
       ...,
       [1.41421356, 1.41421356, 1.4008584 , ..., 0.        , 1.41421356,
        1.41421356],
       [1.41421356, 1.41421356, 1.41421356, ..., 1.41421356, 0.        ,
        1.39880624],
       [1.41421356, 1.41421356, 1.41421356, ..., 1.41421356, 1.39880624,
        0.        ]])

Ahora vamos a definir una funcion que tome como entrada el título de una película y devuelve una lista de las peliculas más similares a esa película de entrada

Para ello, primero tomamos una lista de referencia con los distintos titulos e indices de las peliculas:

In [ ]:
indices = pd.Series(pelis_P.index, index=pelis_P['title']).drop_duplicates()
indices.head()

title
Toy Story                      0
Jumanji                        1
Father of the Bride Part II    4
Heat                           5
Sudden Death                   8
dtype: int64

### 3.2 Funcion de recomendacion

A continuación construimos la función de recomendación. Los pasos que se van a seguir son los siguientes:

- Obtener el índice de la pelicula dado su título
- Obtener la lista con los scores de similitud para esa película con respecto a las demás películas. 
- Ordenar la lista de tuplas en base al score de similitud
- Obtener el top-k de peliculas más similares
- Devolver los títulos que corresponden con los índices de las peliculas más similares


In [ ]:
def rec_pelis(titulo, num_pelis, sim):
    # Indice de la pelicula para el titulo
    idx = indices[titulo]

    # Obtiene los valores de similtud para la pelicula de entrada
    sim_scores = list(enumerate(sim[idx]))

    # Ordena las peliculas a base a los scores de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Scores de las k películas más similares (nótese que dejamos el primer elemento fuera)
    sim_scores = sim_scores[1:num_pelis+1]

    # Indices de las peliculas
    pelis_indices = [i[0] for i in sim_scores]

    # Devuelve las k peliculas mas similares
    return pelis_P['title'].iloc[pelis_indices]

In [ ]:
rec_pelis('Toy Story', 7, sim_cos)

15348    Toy Story 3           
2997     Toy Story 2           
10301    The 40 Year Old Virgin
1071     Rebel Without a Cause 
3057     Man on the Moon       
2157     Indecent Proposal     
10585    Match Point           
Name: title, dtype: object

Aunque las primeras dos entradas de la recomendación parecen bastante acertadas, la tercera o la séptima recomendación parece totalmente inapropiada, sobre todo si tenemos en cuenta que la película de entrada está dirigida al público infantil. 


Recordemos los géneros de nuestra base de datos:

In [ ]:
generos = pelis_P.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
list(pd.unique(generos))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


['Animation',
 'Comedy',
 'Family',
 'Adventure',
 'Fantasy',
 'Action',
 'Crime',
 'Drama',
 'Thriller',
 'Romance',
 'Horror',
 'Science Fiction',
 'Mystery',
 'History',
 'War',
 'Western',
 'Music',
 'Documentary',
 'TV Movie']

### Pregunta 3.3

Qué solución puede plantear para este problema?

**Una posible solución al problema anterior se puede dar  mediante la integración de la variable 'Género' en la función, de modo que las recomendaciones resultantes corespondan con el mismo tipo de género al de la película consultada.**

### Ejercicio 3.4

Escriba su codigo a continuación, donde explore un mejor recomendador que el propuesto arriba. Note que no tenemos más información que la descripcion de las peliculas y su valoracion media. Por ello la evaluacion de la salida es, por el momento, completamente subjetiva (depende de usted). Explique por qué su propuesta es mejor que la que hemos desarrollado hasta el momento.

In [ ]:
def rec_pelis1(titulo, num_pelis, sim):
    # Indice de la pelicula para el titulo
    idx = indices[titulo]

    # Obtiene los valores de similtud para la pelicula de entrada
    sim_scores = list(enumerate(sim[idx]))

    # Ordena las peliculas a base a los scores de similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Scores de las k películas más similares (nótese que dejamos el primer elemento fuera)
    sim_scores = sim_scores[1:num_pelis+1]

    # Indices de las peliculas
    pelis_indices = [i[0] for i in sim_scores]

    # Devuelve las k peliculas mas similares
    return pelis_P[['title','genres']].iloc[pelis_indices]

In [ ]:
retorno=rec_pelis1('Toy Story', 7, sim_cos)

In [ ]:
retorno.head()

,title,genres
15348,Toy Story 3,"[Animation, Family, Comedy]"
2997,Toy Story 2,"[Animation, Comedy, Family]"
10301,The 40 Year Old Virgin,"[Comedy, Romance]"
1071,Rebel Without a Cause,[Drama]
3057,Man on the Moon,"[Comedy, Drama, Romance]"


In [ ]:
genero = list(pelis_P.loc[pelis_P['original_title'] == titulo].genres)[0][0] ## genero de la pelicula de entrada
genero

'Animation'

In [ ]:
for i in range(0,retorno['genres'].shape[0]):
  if genero in retorno['genres'].iloc[i]:
    print(retorno['title'].iloc[i])

Toy Story 3
Toy Story 2


**La recomendación radica en las peliculas con contenido similar que ademas pertenezcan al genero PROPIO de la pelicula que se vió. Por lo anterior, se eliminan sesgos de cara al publico en el que se enfoca la pelicula. Cabe resaltar que al considerar los generos secundarios de la pelicula consultada, la recomendacion desmejora al opcionar peliculas intrinsecamente diferentes.**

### Ejercicio 3.5 

Proponga una metodología, con su respectivo algoritmo, que permita medir, de acuerdo con una métrica de su elección, el nivel de acierto de las recomendaciones.

*Ayuda: considere un sistema de recomendación basado en contenido donde solo hay un usuario (promedio)*

**1. Para cada genero, particionar las peliculas vistas por el usuario en un porcentaje para entrenar y para testear, por ejemplo, 70-30.**

**2. Correr el algoritmo con la partición de entrenamiento.**

**3. Contar porcentualmente el número de recomendaciones que en efecto vió el usuario. La anterior será la métrica del nivel de acierto de las recomendaciones.**